<a href="https://colab.research.google.com/github/Cypher594/RL_damp_cons/blob/main/Design_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
x=df.drop(['amplitude','damping constant'],axis='columns')
y = df['amplitude','damping constant']
# y2= df['damping constant']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.tree import DecisionTreeRegressor
import numpy as np
dt_amp = DecisionTreeRegressor(random_state=42)
dt_amp.fit(x_train, y_train[0])
y_pred_smp = dt_amp.predict(x_test)
dt_amp.score(y_pred_amp,y_test[0])

#Apply grid search for hyper parameters => verma.

# amp_model = keras.Sequential([
#     keras.layers.Dense(3, input_shape=(6,),activation='relu'),
#     keras.layers.Dense(2, activation='relu'),
#     keras.layers.Dense(1, activation = 'relu')
# ])

# amp_model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['mean_squared_error'])
# amp_model.fit(x_train,y_train[0],epochs = 100)

# dc_model = keras.Sequential([
#     keras.layers.Dense(3, input_shape=(6,),activation='relu'),
#     keras.layers.Dense(2, activation='relu'),
#     keras.layers.Dense(1, activation = 'relu')
# ])

# dc_model.compile(optimizer = 'adam', loss='mean_squared_error', metrics=['mean_squared_error'])
# dc_model.fit(x_train,y_train[1],epochs = 100)

In [ ]:
# amp_model.evaluate(x_test,y_test[0])
# dc_model.evaluate(x_test,y_test[1])

In [ ]:
import gym
from gym import Env
from gym import spaces
import numpy as np

class enviro(Env):
    def __init__(self,sensitivity,penetration,frequency,phase):
        # Observation space consists of T1, T2, D, A, sensitvity,penetration,frequency,phase
        self.observation_space = spaces.Box(low=np.array([0,0, 1, 1,0,0,0,0]), high=np.array([60, 25,10, 10,10000,10000,100,1000]), dtype=np.float32)

        # Action space consists of the change in T1 and T2
        self.action_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)

        # Initialize state variables
        self.T1 = 30
        self.T2 = 5
        self.episode_length = 50
        self.sensitivity = sensitivity
        self.penetration = penetration
        self.frequency = frequency
        self.phase = phase
        self.prev_D = 0
        self.prev_A = 0

    def step(self, action):
        # Update T1 and T2 based on action
#         self.T1 = np.clip(self.T1 + action[0], 0, 1)
        self.episode_length = self.episode_length - 1
        self.T1 = self.T1 + action[0]
        if(self.T1<0):
            self.T1 = 0
        elif(self.T1>60):
            self.T1=60

        self.T2 = self.T2 + action[1]
        if(self.T2<0):
            self.T2 = 0
        elif(self.T2>25):
            self.T2=25

        ip = [self.sensitivity,self.penetration,self.frequency,self.phase,self.T1,self.T2]

        # Calculate derived variables
        # D = dc_model.predict(ip)
        # A = amp_model.predict(ip)
        D = (self.T1)*(self.T2)
        A = (100-self.T1)*(100-self.T2)
        # Reward is based on the difference between current and previous values of D and A
        reward = (D - self.prev_D) - 5*(A - self.prev_A)

        # Save current values of D and A for the next time step
        self.prev_D = D
        self.prev_A = A

        if(self.episode_length<=0):
            done = True
        else:
            done = False

        # Information is an empty dictionary
        info = {}

        # Return the observation, reward, done, and information
        return np.array([self.T1, self.T2, D, A,self.sensitivity,self.penetration,self.frequency,self.phase]), reward, done, info

    def reset(self):
        # Reset T1 and T2 to their initial values
        self.T1 = 30
        self.T2 = 5

        # Calculate initial values of D and A
        self.prev_D = 0
        self.prev_A = 0

        self.episode_length = 50

        # Return the initial observation
        return np.array([self.T1, self.T2, self.prev_D, self.prev_A, self.sensitivity, self.penetration, self.frequency, self.phase])

    def state_display(self):
        print("T1 =>",self.T1,"\n")
        print("T2 =>",self.T2,"\n")

In [ ]:
env = enviro()
episodes = 10
for episode in range(1,episodes+1):
    score = 0
    state = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        n_state,reward,done,info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))


Episode:1 Score:-68.74852343819293
Episode:2 Score:-325.772513548408
Episode:3 Score:54.943047341144776
Episode:4 Score:-150.006860131228
Episode:5 Score:149.60004736639894
Episode:6 Score:-974.9924600501384
Episode:7 Score:336.19862847119043
Episode:8 Score:465.8564933671187
Episode:9 Score:-606.9689419576086
Episode:10 Score:70.22345087887379


In [ ]:
# q learning withhout library
discount_factor = 0.95
eps = 0.5
eps_decay_factor = 0.999
learning_rate = 0.8
num_episodes = 500
q_table = np.zeros([4,2])
for i in range(num_episodes):
    state = env.reset()
    eps *= eps_decay_factor
    done = False
    while not done:
        if np.random.random() < eps or np.sum(q_table[state, :]) == 0:
            action = np.random.randint(0, 2)
        else:
            action = np.argmax(q_table[state, :])
        new_state, reward, done, _ = env.step(action)
        q_table[state, action] += reward + learning_rate *(discount_factor * np.max(q_table[new_state, :]) - q_table[state, action])
        state = new_state

IndexError: index 30 is out of bounds for axis 0 with size 4

In [ ]:
import stable_baselines3
from stable_baselines3 import DQN
input_new = [0.3,0,1.2,6]
env= enviro(input_new)
model = DQN('MlpPolicy',env,verbose=1,tensorboard_log=log_path)
model.learn(total_timesteps=20000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: ignored

In [ ]:
from stable_baselines3 import PPO
env= enviro(0.3,0,1.2,6)

model = PPO('MlpPolicy',env,verbose=1)
model.learn(total_timesteps=20000)          #20000 is a small number btw

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 51       |
|    ep_rew_mean     | -63.9    |
| time/              |          |
|    fps             | 197      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 51          |
|    ep_rew_mean          | 159         |
| time/                   |             |
|    fps                  | 231         |
|    iterations           | 2           |
|    time_elapsed         | 17          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.021148909 |
|    clip_fraction        | 0.294       |
|    clip_range           | 0.2         |
|    entropy_loss  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 51          |
|    ep_rew_mean          | 610         |
| time/                   |             |
|    fps                  | 278         |
|    iterations           | 11          |
|    time_elapsed         | 80          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010088727 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.56       |
|    explained_variance   | 0.327       |
|    learning_rate        | 0.0003      |
|    loss                 | 3.98e+03    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00659    |
|    std                  | 0.874       |
|    value_loss           | 1.23e+04    |
-----------------------------------------
-----------------------------------------
| rollout/                |       